In [1]:
import ot
import numpy as np
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [63]:
numpoints = 3
a=[[1.,1.,1.]]
b=[[1.,1.,1.]]
a_positions = np.array([[[0.,0.],[1.,1.],[1.,2.]]])
b_positions = np.array([[[1.,1.1],[3.,0.23],[0.,4.1]]])
M=np.array([[np.linalg.norm(a-b) for a in a_positions[0]] for b in b_positions[0]])
result=ot.emd(a[0],b[0],M,log=True)
plan = result[0]
cost = result[1]['cost']

In [77]:
def plan_wrapper(Ms):
    result=[ot.emd(np.ones(len(M[0])),np.ones(len(M[1])),M) for M in Ms]
    return np.array(result)

In [78]:
plan_wrapper([M])

array([[[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]]])

In [ ]:
def jacobian(y, x):
  y_flat = tf.reshape(y, (-1,))
  jacobian_flat = tf.stack(
      [tf.gradients(y_i, x)[0] for y_i in tf.unstack(y_flat)])
  return tf.reshape(jacobian_flat, y.shape.concatenate(x.shape))

In [1]:
import ot
import numpy as np
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

def plan_python(Ms):
    result=[ot.emd(np.ones(len(M[0])),np.ones(len(M[1])),M) for M in Ms]
    return np.array(result)

def myground_dist_func(a_positions, b_positions, numpoints=3):
    tiled_b = tf.tile(b_positions,[1,numpoints,1])
    repeated_a = tf.reshape(a_positions, [1,-1,1,2])    # Convert to a len(yp) x 1 matrix.
    repeated_a = tf.tile(repeated_a, [1,1,numpoints,1])  # Create multiple columns.
    repeated_a = tf.reshape(repeated_a, [1,-1,2])  

    ground_dist = tf.norm(repeated_a-tiled_b,axis=-1)
    ground_dist = tf.reshape(ground_dist,[1,numpoints,numpoints])
    
    return ground_dist



a_positions = np.array([[[0.,0.],[1.,1.],[1.,2.]]])
b_positions = np.array([[[1.,1.1],[3.,0.23],[0.,4.1]]])
tensor_a_positions = tf.Variable(a_positions)
tensor_b_positions = tf.Variable(b_positions)

with tf.Session() as sess:
    sess.run(tensor_a_positions.initializer)
    sess.run(tensor_b_positions.initializer)
    ground_dist_tensor = myground_dist_func(tensor_a_positions, tensor_b_positions)
    plan_tensor = tf.py_func(func=plan_python, inp=[ground_dist_tensor],Tout=tf.float64)
    print("Transport plan: \n",sess.run(plan_tensor))
    
    my_calc_loss_tensor = tf.linalg.trace(tf.matmul(ground_dist_tensor,tf.transpose(plan_tensor, perm=[0,2,1])))
    print("\nEMD:\n",my_calc_loss_tensor.eval())
    
    gradient_tensor = tf.gradients(my_calc_loss_tensor,tensor_b_positions)
    print("\nGradient:\n",sess.run(gradient_tensor))

Transport plan: 
 [[[0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]]]

EMD:
 [5.43474442]

Gradient:
 [array([[[ 0.        ,  1.        ],
        [ 0.997074  ,  0.07644234],
        [-0.42993358,  0.90286052]]])]


In [1]:
import ot
import numpy as np
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

def plan_python(Ms):
    result=[ot.emd(np.ones(len(M[0])),np.ones(len(M[1])),M) for M in Ms]
    return np.array(result)

def myground_dist_func(a_positions, b_positions,numpoints=3):
    tiled_b = tf.tile(b_positions,[1,numpoints,1])
    repeated_a = tf.reshape(a_positions, [1,-1,1,2])    # Convert to a len(yp) x 1 matrix.
    repeated_a = tf.tile(repeated_a, [1,1,numpoints,1])  # Create multiple columns.
    repeated_a = tf.reshape(repeated_a, [1,-1,2])  

    ground_dist = tf.norm(repeated_a-tiled_b,axis=-1)
    ground_dist = tf.reshape(ground_dist,[1,numpoints,numpoints])
    
    return ground_dist

def emd_loss(y_true, y_pred):
    ground_dist_tensor = myground_dist_func(y_true, y_pred)
    plan_tensor = tf.py_func(func=plan_python, inp=[ground_dist_tensor],Tout=tf.float64)
    
    my_calc_loss_tensor = tf.linalg.trace(tf.matmul(ground_dist_tensor,tf.transpose(plan_tensor, perm=[0,2,1])))
    return my_calc_loss_tensor


y_true_nparr = np.array([[[0.,0.],[1.,1.],[1.,2.]]])
y_pred_nparr = np.array([[[1.,1.1],[3.,0.23],[0.,4.1]]])
y_true = tf.Variable(y_true_nparr)
y_pred = tf.Variable(y_pred_nparr)

with tf.Session() as sess:
    sess.run(y_true.initializer)
    sess.run(y_pred.initializer)
    loss = emd_loss(y_true,y_pred)
    gradients = tf.gradients(loss,y_pred)
    print(sess.run(emd_loss(y_true,y_pred)))
    print(sess.run(gradients))

[5.43474442]
[array([[[ 0.        ,  1.        ],
        [ 0.997074  ,  0.07644234],
        [-0.42993358,  0.90286052]]])]


In [89]:
myloss = tf.Variable(np.array([cost]))
#myplan = tf.Variable(np.array([plan]))
tensor_a_positions = tf.Variable(a_positions)
tensor_b_positions = tf.Variable(b_positions)



with tf.Session() as sess:
    sess.run(myloss.initializer)
    #sess.run(myplan.initializer)
    sess.run(tensor_a_positions.initializer)
    sess.run(tensor_b_positions.initializer)
    #sess.run(myotherthing.initializer)
    #sess.run(myprod.initializer)
    ground_dist_tensor = myground_dist_func(tensor_a_positions, tensor_b_positions)
    myplan = tf.py_func(func=plan_wrapper, inp=[ground_dist_tensor],Tout=tf.float64)
    print(sess.run(myplan))
    #my_calc_loss_tensor = tf.linalg.trace(tf.matmul(ground_dist_tensor,tf.transpose(myplan)))
    my_calc_loss_tensor = tf.linalg.trace(tf.matmul(ground_dist_tensor,tf.transpose(myplan, perm=[0,2,1])))
    print(my_calc_loss_tensor.eval())
    print(sess.run(ground_dist_tensor))
    jac_grounddist = jacobian(ground_dist_tensor,tensor_b_positions)
    jac_grounddist = tf.transpose(jac_grounddist, perm=[3,4,5,0,1,2])
    print(jac_grounddist.shape)
#     #my_calc_derivative = tf.linalg.trace(tf.matmul(tf.transpose(jac_grounddist, perm=[3,4,5,0,1,2]),tf.reshape(tf.transpose(myplan, perm=[0,2,1]),[1,1,1,1,3,3])))
    my_tensordot = tf.transpose(tf.tensordot(jac_grounddist,myplan,[[5],[1]]),perm=[0,1,2,3,5,4,6])
    my_tensordot = tf.trace(my_tensordot)
    print(sess.run(my_tensordot))
#     #print(sess.run( tf.linalg.trace(tf.matmul(tf.gradients(ground_dist_tensor,tensor_b_positions)[0],tf.transpose(myplan)))))
    print(sess.run(tf.gradients(my_calc_loss_tensor,tensor_b_positions)))
    #print(my_calc_loss_tensor.eval())
    #print(tf.gradients(myprod,myotherthing)[0][0].eval())

[[[0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]]]
[5.43474442]
[[[1.48660687 3.00880375 4.1       ]
  [0.1        2.14310522 3.25729949]
  [0.9        2.67074896 2.32594067]]]
(1, 3, 2, 1, 3, 3)
[[[[[ 0.        ]]

   [[ 1.        ]]]


  [[[ 0.997074  ]]

   [[ 0.07644234]]]


  [[[-0.42993358]]

   [[ 0.90286052]]]]]
[array([[[ 0.        ,  1.        ],
        [ 0.997074  ,  0.07644234],
        [-0.42993358,  0.90286052]]])]


In [45]:
from tensorflow.python.ops.parallel_for.gradients import jacobian

ImportError: No module named 'tensorflow.python.ops.parallel_for'

In [40]:
v = tf.Variable([1, 2])
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(v.initializer)
    #sess.run(init)
    # Usage passing the session explicitly.
    print(v.eval(sess))
    # Usage with the default session.  The 'with' block
    # above makes 'sess' the default session.
    print(v.eval())

[1 2]
[1 2]
